# Claude CLI - References et Contexte

**Module :** Vibe-Coding / Claude Code / Notebooks CLI  
**Niveau :** Intermediaire  
**Duree :** 25 min  
**Prerequis :** Notebooks 01-02 completes

## Objectifs d'Apprentissage

- [ ] Utiliser les @-mentions pour referencer des fichiers
- [ ] Specifier des plages de lignes (@file:10-20)
- [ ] Referencer des repertoires entiers
- [ ] Comprendre le role de CLAUDE.md
- [ ] Analyser un mini-projet complet

---

## Pourquoi les References ?

Les @-mentions sont essentielles pour le **Vibe-Coding** car elles permettent :

1. **Contexte precis** : Claude comprend exactement quel code vous discutez
2. **Moins de copier-coller** : Plus besoin de coller manuellement le code
3. **References dynamiques** : Le fichier est relu a chaque appel (toujours a jour)
4. **Economie de tokens** : Les plages de lignes limitent le contexte au strict necessaire

> **Note importante :** En mode interactif (`claude` sans `-p`), les @-mentions sont auto-completees avec Tab. En mode CLI avec `-p`, vous devez specifier le chemin complet depuis le repertoire courant.

## 1. Configuration

In [ ]:
import sys
import subprocess
import os

sys.path.insert(0, 'helpers')
from claude_cli import run_claude, verify_installation, print_response

# Repertoire des exemples
EXAMPLES_DIR = os.path.join(os.getcwd(), 'examples', 'sample_project')
print(f"Repertoire exemples: {EXAMPLES_DIR}")
print(f"Existe: {os.path.exists(EXAMPLES_DIR)}")

## 2. References de Fichiers (@fichier)

Claude Code permet de referencer des fichiers directement dans vos prompts avec la syntaxe `@fichier` :

```bash
# Fichier dans le repertoire courant
claude -p "Explique @main.py"

# Fichier dans un sous-repertoire
claude -p "Analyse @src/utils.py"
```

Le contenu du fichier est automatiquement inclus dans le contexte de Claude.

### Syntaxes supportees

| Syntaxe | Description | Exemple |
|---------|-------------|---------|
| `@fichier` | Fichier entier | `@main.py` |
| `@chemin/fichier` | Avec chemin relatif | `@src/utils.py` |
| `@fichier:L1-L2` | Plage de lignes | `@main.py:10-25` |
| `@dossier/` | Repertoire | `@src/` |

> **Attention en mode CLI :** Avec `claude -p`, les @-mentions fonctionnent depuis le repertoire courant du terminal. Assurez-vous d'etre dans le bon repertoire ou utilisez des chemins relatifs corrects.

> **Tip :** En mode interactif, tapez `@` puis Tab pour l'auto-completion des chemins.

**Note :** En mode CLI avec `-p`, les @-mentions fonctionnent depuis le repertoire courant.

In [ ]:
# Voir le contenu du fichier main.py
main_path = os.path.join(EXAMPLES_DIR, 'main.py')
with open(main_path, 'r') as f:
    content = f.read()
print(content)

In [ ]:
# Analyser le fichier avec Claude
# En mode script, on passe le contenu directement
with open(main_path, 'r') as f:
    main_content = f.read()

stdout, stderr, code = run_claude(
    f"Analyse ce fichier Python et resume son objectif en 3 points:\n\n{main_content}"
)
print_response(stdout, stderr, code)

## 3. Plages de Lignes (@fichier:debut-fin)

Pour referencer seulement une partie d'un fichier, utilisez la notation `:ligne-ligne` :

```bash
# Lignes 10 a 20 (incluses)
claude -p "Explique @utils.py:10-20"

# A partir de la ligne 5 jusqu'a la fin
claude -p "Explique @utils.py:5-"

# Du debut jusqu'a la ligne 15
claude -p "Explique @utils.py:-15"

# Une seule ligne
claude -p "Que fait @utils.py:42"
```

### Cas d'usage typiques

| Situation | Commande |
|-----------|----------|
| Analyser une fonction specifique | `@utils.py:15-45` |
| Comprendre les imports | `@main.py:-10` |
| Debugger une erreur a la ligne 42 | `@code.py:40-50` |
| Revoir la fin d'un fichier | `@log.py:100-` |

> **Economie de tokens :** Sur un fichier de 500 lignes, referencer seulement les lignes 100-150 reduit le contexte de 90%, ce qui accelere les reponses et reduit les couts.

In [ ]:
# Voir utils.py
utils_path = os.path.join(EXAMPLES_DIR, 'utils.py')
with open(utils_path, 'r') as f:
    lines = f.readlines()

# Afficher avec numeros de ligne
for i, line in enumerate(lines, 1):
    print(f"{i:3}: {line}", end='')

In [ ]:
# Analyser seulement la fonction calculate_statistics (lignes 15-50 environ)
relevant_lines = ''.join(lines[14:50])  # Indices Python (0-based)

stdout, stderr, code = run_claude(
    f"Explique cette fonction et identifie d'eventuels bugs:\n\n{relevant_lines}"
)
print_response(stdout, stderr, code)

## 4. Reference de Repertoires (@dossier/)

Pour donner a Claude une vue d'ensemble d'un projet, referencez un repertoire entier :

```bash
claude -p "Analyse la structure de @src/"
```

### Comportement

Quand vous referencez un repertoire :

1. Claude recoit la **liste des fichiers** (pas leur contenu complet)
2. Il peut ensuite **demander** le contenu de fichiers specifiques si necessaire
3. Les sous-repertoires sont inclus recursivement

### Limitations importantes

| Limitation | Description |
|------------|-------------|
| **Taille** | Les tres grands repertoires (>100 fichiers) peuvent depasser les limites de contexte |
| **Binaires** | Les fichiers binaires (images, .exe) sont ignores |
| **Git** | Le dossier `.git/` est automatiquement exclu |
| **node_modules** | Les dependances volumineuses sont generalement exclues |

> **Bonnes pratiques :**
> - Preferez referencer des sous-repertoires specifiques (`@src/models/`)
> - Pour un gros projet, combinez avec un fichier CLAUDE.md pour le contexte

Claude recevra la liste des fichiers et pourra demander leur contenu.

In [ ]:
# Lister la structure du projet exemple
def list_files(directory, prefix=""):
    """Liste recursivement les fichiers d'un repertoire."""
    items = []
    for item in sorted(os.listdir(directory)):
        path = os.path.join(directory, item)
        if os.path.isfile(path):
            items.append(f"{prefix}{item}")
        elif os.path.isdir(path) and not item.startswith('.'):
            items.append(f"{prefix}{item}/")
            items.extend(list_files(path, prefix + "  "))
    return items

print("Structure du projet exemple:")
for f in list_files(EXAMPLES_DIR):
    print(f"  {f}")

In [ ]:
# Analyser le projet complet
# Collecter tous les fichiers Python
project_content = []

for root, dirs, files in os.walk(EXAMPLES_DIR):
    for file in files:
        if file.endswith('.py'):
            filepath = os.path.join(root, file)
            relpath = os.path.relpath(filepath, EXAMPLES_DIR)
            with open(filepath, 'r') as f:
                content = f.read()
            project_content.append(f"=== {relpath} ===\n{content}")

full_project = "\n\n".join(project_content)
print(f"Taille totale: {len(full_project)} caracteres")

In [ ]:
# Demander une analyse globale
stdout, stderr, code = run_claude(
    f"""Analyse ce projet Python et donne:
1. L'architecture generale
2. Les points forts
3. Les ameliorations possibles

Projet:
{full_project}"""
)
print_response(stdout, stderr, code)

## 5. Fichier CLAUDE.md - Le Contexte Permanent

Le fichier `CLAUDE.md` a la racine d'un projet est **automatiquement lu** par Claude Code a chaque interaction. C'est votre moyen de donner des instructions persistantes.

### Pourquoi c'est important ?

```
Sans CLAUDE.md:
┌─────────────────────────────────────────┐
│ "Corrige le bug dans auth.py"           │
│                                          │
│ Claude: "Quel framework utilisez-vous?" │
│ Vous: "FastAPI avec PostgreSQL"          │
│ ...repetition a chaque session...        │
└─────────────────────────────────────────┘

Avec CLAUDE.md:
┌─────────────────────────────────────────┐
│ "Corrige le bug dans auth.py"           │
│                                          │
│ Claude connait deja:                     │
│ - Framework: FastAPI + PostgreSQL        │
│ - Style: PEP 8, Google docstrings        │
│                                          │
│ -> Reponse directe et adaptee            │
└─────────────────────────────────────────┘
```

### Contenu recommande

| Section | Description |
|---------|-------------|
| **Description** | But du projet en 2-3 phrases |
| **Structure** | Arborescence des dossiers principaux |
| **Commandes** | Scripts de build, test, deploy |
| **Conventions** | Style de code, nommage, patterns |

### Emplacements

Claude cherche le fichier dans cet ordre :
1. `./CLAUDE.md` (racine du projet)
2. `~/.claude/CLAUDE.md` (configuration globale)

> **Tip :** Creez un CLAUDE.md global pour vos preferences communes (langue, style), et des CLAUDE.md specifiques par projet pour les conventions techniques.

Claude le lit automatiquement quand vous travaillez dans le projet.

In [ ]:
# Voir notre exemple de CLAUDE.md
claude_md_path = os.path.join(os.getcwd(), 'examples', 'CLAUDE.md')
with open(claude_md_path, 'r') as f:
    claude_md = f.read()

print(claude_md)

In [ ]:
# Generer un CLAUDE.md pour un nouveau projet
stdout, stderr, code = run_claude(
    f"""A partir de ce code, genere un fichier CLAUDE.md complet:

{full_project}

Le CLAUDE.md doit inclure:
- Description du projet
- Structure des fichiers
- Commandes principales
- Conventions de code"""
)
print_response(stdout, stderr, code)

## 6. Exercices avec le Projet Exemple

In [ ]:
# EXERCICE 1 : Analysez la fonction format_report de utils.py
# et proposez des ameliorations

# Etape 1 : Extraire les lignes de format_report (lignes 60-81)
# format_report_lines = ''.join(lines[59:81])

# Etape 2 : Demander des ameliorations
# stdout, stderr, code = run_claude(
#     f"Voici la fonction format_report. Propose 3 ameliorations:\n\n{format_report_lines}"
# )
# print_response(stdout, stderr, code)

print("Decommentez le code ci-dessus pour executer l'exercice")

In [ ]:
# EXERCICE 2 : Demandez a Claude de generer des tests supplementaires
# pour la fonction normalize_data

# Localisez d'abord la fonction (lignes 105-125)
# normalize_lines = ''.join(lines[104:125])

# Demandez des tests pytest couvrant:
# 1. Cas normal (liste de nombres)
# 2. Liste vide
# 3. Valeurs identiques (range = 0)

# stdout, stderr, code = run_claude(
#     f"Genere des tests pytest pour cette fonction:\n\n{normalize_lines}"
# )
# print_response(stdout, stderr, code)

print("Decommentez le code ci-dessus pour executer l'exercice")

In [ ]:
# EXERCICE 3 : Demandez une revue de code complete du projet
# avec focus sur la securite et les performances

# Utilisez full_project qui contient tout le code
# stdout, stderr, code = run_claude(
#     f"""Effectue une revue de code de ce projet Python.
# Focus sur:
# 1. Problemes de securite potentiels
# 2. Optimisations de performance
# 3. Bonnes pratiques manquantes
# 
# Projet:
# {full_project}"""
# )
# print_response(stdout, stderr, code)

print("Decommentez le code ci-dessus pour executer l'exercice")

## 7. Resume

Dans ce notebook, nous avons appris :

| Concept | Syntaxe | Usage |
|---------|---------|-------|
| **Fichier entier** | `@fichier` | Inclure tout le contenu |
| **Plage de lignes** | `@fichier:L1-L2` | Cibler une section precise |
| **Repertoire** | `@dossier/` | Vue d'ensemble du projet |
| **CLAUDE.md** | Fichier a la racine | Contexte permanent automatique |

### Erreurs courantes et solutions

| Erreur | Cause | Solution |
|--------|-------|----------|
| "File not found" | Chemin relatif incorrect | Verifiez le repertoire courant |
| Contenu tronque | Fichier trop gros | Utilisez les plages de lignes |
| @-mention ignoree | Pas d'espace avant le @ | Ecrivez `texte @fichier` |

### Points cles a retenir

1. Les @-mentions economisent du temps et des tokens
2. Les plages de lignes sont essentielles pour les gros fichiers
3. CLAUDE.md est votre meilleur allie pour le contexte projet
4. En mode `-p`, les chemins sont relatifs au repertoire courant

### Prochaine etape

Dans le notebook suivant, nous decouvrirons les **agents** (Explore, Plan) et les **subagents**.

-> [04-Claude-CLI-Agents.ipynb](04-Claude-CLI-Agents.ipynb)